# DSCI 417 - Homework 04
**Malcolm Nichols**

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

spark = SparkSession.builder.getOrCreate()

## Load Diamond Data

In [0]:
di_schema = (
  'carat DOUBLE, cut STRING, color STRING, clarity STRING, depth DOUBLE,'
  'table DOUBLE, price INTEGER, x DOUBLE, y DOUBLE, z DOUBLE'
)

diamonds = (
  spark.read
  .option('delimiter', '\t')
  .option('header', True)
  .schema(di_schema)
  .csv('/FileStore/tables/diamonds.txt')
)

diamonds.printSchema()

root
-- carat: double (nullable = true)
-- cut: string (nullable = true)
-- color: string (nullable = true)
-- clarity: string (nullable = true)
-- depth: double (nullable = true)
-- table: double (nullable = true)
-- price: integer (nullable = true)
-- x: double (nullable = true)
-- y: double (nullable = true)
-- z: double (nullable = true)

## Problem 1: Grouping By Cut

In [0]:
def rank_cut(x):
    cut_level = {'Fair' : 1, 'Good': 2, 'Very Good' : 3, 'Premium' : 4, 'Ideal' : 5}
    if x in cut_level.keys():
        return cut_level[x]
    
spark.udf.register('RANK_CUT', rank_cut)

Out[3]: <function __main__.rank_cut(x)>

In [0]:
(
    diamonds
    .groupBy('cut')
    .agg(
        expr('COUNT(*) AS n_diamonds'),
        expr('ROUND(MEAN(price)) AS avg_price'),
        expr('ROUND(MEAN(carat),2) AS avg_carat'),
        expr('ROUND(MEAN(depth),2) AS avg_depth'),
        expr('ROUND(MEAN(table),2) AS avg_table')
    )
    .sort(expr('RANK_CUT(cut)'))
    .show()
)

+---------+----------+---------+---------+---------+---------+
 cut|n_diamonds|avg_price|avg_carat|avg_depth|avg_table|
+---------+----------+---------+---------+---------+---------+
 Fair| 1610| 4359.0| 1.05| 64.04| 59.05|
 Good| 4906| 3929.0| 0.85| 62.37| 58.69|
Very Good| 12082| 3982.0| 0.81| 61.82| 57.96|
 Premium| 13791| 4584.0| 0.89| 61.26| 58.75|
 Ideal| 21551| 3458.0| 0.7| 61.71| 55.95|
+---------+----------+---------+---------+---------+---------+

## Problem 2: Filtering based on Carat Size

In [0]:
for n in range(6):
    diamond_count = (
        diamonds
        .filter(col('carat') >= n)
        .filter(col('carat') < n + 1)
        .count()
    )
    
    print(f'The number of diamonds with carat size in range [{n}, {n+1}) is {diamond_count}.')

The number of diamonds with carat size in range [0, 1) is 34880.
The number of diamonds with carat size in range [1, 2) is 16906.
The number of diamonds with carat size in range [2, 3) is 2114.
The number of diamonds with carat size in range [3, 4) is 34.
The number of diamonds with carat size in range [4, 5) is 5.
The number of diamonds with carat size in range [5, 6) is 1.

## Problem 3: Binning by Carat Size

In [0]:
def carat_bin(x):
    bins = ['[0,1)', '[1,2)', '[2,3)', '[3,4)', '[4,5)', '[5,6)']
    return bins[int(x)]

spark.udf.register('CARAT_BIN', carat_bin)

Out[6]: <function __main__.carat_bin(x)>

In [0]:
(
    diamonds
    .withColumn('carat_bin', expr('CARAT_BIN(carat)'))
    .groupBy('carat_bin')
    .agg(
        expr('COUNT(*) AS n_diamonds'),
        expr('ROUND(MEAN(price)) AS avg_price')
    )
    .sort('carat_bin')
    .show()
)

+---------+----------+---------+
carat_bin|n_diamonds|avg_price|
+---------+----------+---------+
 [0,1)| 34880| 1633.0|
 [1,2)| 16906| 7288.0|
 [2,3)| 2114| 14847.0|
 [3,4)| 34| 14309.0|
 [4,5)| 5| 16458.0|
 [5,6)| 1| 18018.0|
+---------+----------+---------+

## Load IMDB Data

In [0]:
movies = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .csv('/FileStore/tables/imdb/movies.txt')
)

movies.printSchema()

root
-- imdb_title_id: string (nullable = true)
-- title: string (nullable = true)
-- year: string (nullable = true)
-- genre: string (nullable = true)
-- duration: string (nullable = true)
-- country: string (nullable = true)
-- language: string (nullable = true)

In [0]:
names = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .csv('/FileStore/tables/imdb/names.txt')
)

names.printSchema()

root
-- imdb_name_id: string (nullable = true)
-- name: string (nullable = true)
-- birth_name: string (nullable = true)
-- height: string (nullable = true)
-- bio: string (nullable = true)
-- date_of_birth: string (nullable = true)
-- date_of_death: string (nullable = true)

In [0]:
title_principals = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .csv('/FileStore/tables/imdb/title_principals.txt')
)

title_principals.printSchema()

root
-- imdb_title_id: string (nullable = true)
-- ordering: string (nullable = true)
-- imdb_name_id: string (nullable = true)
-- category: string (nullable = true)
-- characters: string (nullable = true)

In [0]:
ratings = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .csv('/FileStore/tables/imdb/ratings.txt')
)

ratings.printSchema()

root
-- imdb_title_id: string (nullable = true)
-- rating: string (nullable = true)
-- total_votes: string (nullable = true)

In [0]:
print(movies.count())
print(names.count())
print(title_principals.count())
print(ratings.count())

85855
297710
835513
85855

## Problem 4: Number of Appearances by Actor

In [0]:
(
    title_principals
    .filter(expr('category == "actor" OR category == "actress"'))
    .groupBy('imdb_name_id')
    .agg(expr('COUNT(*) AS appearances'))
    .join(names, 'imdb_name_id', 'left')
    .select('name', 'appearances')
    .sort('appearances', ascending=False)
    .show(16)
)

+-----------------+-----------+
 name|appearances|
+-----------------+-----------+
 Mohanlal| 163|
 Amitabh Bachchan| 142|
 Mammootty| 140|
 Eric Roberts| 133|
 John Wayne| 132|
 Gérard Depardieu| 130|
 Prakash Raj| 125|
 Akshay Kumar| 115|
 Michael Madsen| 107|
 Andy Lau| 102|
Catherine Deneuve| 101|
 Anupam Kher| 99|
 Brahmanandam| 99|
 Ajay Devgn| 94|
 Michael Caine| 94|
 Christopher Lee| 93|
+-----------------+-----------+
only showing top 16 rows

In [0]:
(
    title_principals
    .filter(expr('category == "director"'))
    .join(ratings, 'imdb_title_id', 'left')
    .groupBy('imdb_name_id')
    .agg(
        expr('COUNT(*) AS num_films'),
        expr('SUM(total_votes) AS total_votes'),
        expr('ROUND(MEAN(rating),2) AS avg_rating')
    )
    .filter(col('total_votes') >= 1000000)
    .filter(col('num_films') >= 5)
    .join(names, 'imdb_name_id', 'left')
    .select('name', 'num_films', 'total_votes', 'avg_rating')
    .sort('avg_rating', ascending=False)
    .show(16, truncate=False)
)

+---------------------+---------+-----------+----------+
name |num_films|total_votes|avg_rating|
+---------------------+---------+-----------+----------+
Christopher Nolan |11 |1.1653144E7|8.22 |
Lee Unkrich |5 |3329612.0 |8.14 |
Hayao Miyazaki |12 |2254496.0 |8.01 |
Sergio Leone |7 |1720654.0 |7.93 |
Quentin Tarantino |14 |9460772.0 |7.93 |
Stanley Kubrick |13 |4232356.0 |7.78 |
David Fincher |10 |6944421.0 |7.76 |
Sam Mendes |10 |3067512.0 |7.73 |
Wes Anderson |9 |2173090.0 |7.61 |
Alejandro G. Iñárritu|7 |2067540.0 |7.61 |
Peter Jackson |13 |7304418.0 |7.58 |
Brad Bird |6 |2294748.0 |7.57 |
Alfonso Cuarón |8 |2078975.0 |7.54 |
Andrew Stanton |5 |2649551.0 |7.52 |
Bong Joon Ho |8 |1172684.0 |7.51 |
Akira Kurosawa |32 |1061519.0 |7.51 |
+---------------------+---------+-----------+----------+
only showing top 16 rows

## Problem 6: Actors Appearing in Horror Films

In [0]:
horror_films = movies.filter(expr('genre LIKE "%Horror%"'))
print(horror_films.count())

9557

In [0]:
(
    title_principals
    .filter(expr('category == "actor" OR category == "actress"'))
    .join(horror_films, 'imdb_title_id', 'semi')
    .groupBy('imdb_name_id')
    .agg(expr('COUNT(*) AS num_films'))
    .join(names, 'imdb_name_id', 'left')
    .select('name', 'num_films')
    .sort('num_films', ascending=False)
    .show(16)
)

+----------------+---------+
 name|num_films|
+----------------+---------+
 Christopher Lee| 56|
 Peter Cushing| 47|
 Boris Karloff| 46|
 John Carradine| 43|
 Bela Lugosi| 38|
 Vincent Price| 34|
 Lance Henriksen| 33|
 Eric Roberts| 29|
 Lon Chaney Jr.| 28|
 Bill Moseley| 27|
 Tony Todd| 27|
 Paul Naschy| 26|
Donald Pleasence| 26|
 Robert Englund| 23|
 Brad Dourif| 23|
 Sergey A.| 23|
+----------------+---------+
only showing top 16 rows